In [1]:
stock = 'WDC'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    75.989845
2018-07-23    75.732048
2018-07-24    74.472824
2018-07-25    74.988411
2018-07-26    76.436028
2018-07-27    70.526588
2018-07-30    69.435921
2018-07-31    69.554901
2018-08-01    68.642708
2018-08-02    67.938728
2018-08-03    67.383476
2018-08-06    67.720596
2018-08-07    67.175262
2018-08-08    66.481201
2018-08-09    65.549171
2018-08-10    64.240372
2018-08-13    63.675209
2018-08-14    64.438675
2018-08-15    63.407497
2018-08-16    64.508080
2018-08-17    64.081726
2018-08-20    63.476902
2018-08-21    64.339523
2018-08-22    63.476902
2018-08-23    62.812584
2018-08-24    63.268688
2018-08-27    63.923088
2018-08-28    64.121391
2018-08-29    62.852249
2018-08-30    62.743179
                ...    
2018-09-17    56.694931
2018-09-18    57.607124
2018-09-19    57.894665
2018-09-20    59.540581
2018-09-21    60.204899
2018-09-24    59.411682
2018-09-25    59.173721
2018-09-26    58.439999
2018-09-27    58.279999
2018-09-28    58.540001
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


54.83681240789632
[55.81616457 54.6308188  54.42289789 54.67161663 54.64256415]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")